# Drzewo klasyfikacyjne

Wrócimy dziś do zbioru churn z projektu regresji logistycznej. Spróbujemy stworzyć rozwiązanie dla umów bezterminowych.

Zbiór danych: https://www.kaggle.com/datasets/mehmetsabrikunt/internet-service-churn


In [1]:
from  sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split

In [2]:
# Wyznaczenie entropii
def entropia(y: int, n: int):
    t = y+n
    return - (y/t * math.log(y/t,2) +n/t * math.log(n/t,2))

In [3]:
# Wyznaczenie indeksu giniego
def gini_index(y: int, n: int):
    t = y+n
    return 1- ((y/t)**2 + (n/t)**2)

In [4]:
entropia(100,0.0001)

2.137424295738942e-05

In [5]:
gini_index(100,0)

0.0

In [6]:
entropia(50,50)

1.0

In [7]:
gini_index(50,50)

0.5

In [8]:
# puść ten kod, 
# jeżeli wywołujesz plik  w folderze rozwiąznaia, 
# a ramka danych znajduje się w folderze data
import os 
os.chdir('../')

In [9]:
# Pobranie danych
os.chdir('../')

df = pd.read_csv('data/internet_service_churn.csv')

In [15]:
# nagłówek
df.head()

,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,15,1,0,11.95,25,0.14,0,8.4,2.3,0,0
1,18,0,0,8.22,0,NaN,0,0.0,0.0,0,1
2,23,1,0,8.91,16,0.00,0,13.7,0.9,0,1
3,27,0,0,6.87,21,NaN,1,0.0,0.0,0,1
4,34,0,0,6.39,0,NaN,0,0.0,0.0,0,1


In [16]:
# usunięcie braków danych dla download / upload 
df = df[~(df['download_avg'].isna())]

In [17]:
# Sworzenie zmiennej czy umowa terminowa
df['is_fixed_term'] = (~(df['reamining_contract'].isna())).astype(int)

In [18]:
# Stworzenie zbioru dla umów beterminowych.

df_indefinite = df[df['is_fixed_term']==0]

In [19]:
# liczba odejść w umowach bezterminowych
df_indefinite.churn.value_counts()

churn
1    19719
0     1799
Name: count, dtype: int64

In [22]:
# Udział
df_indefinite.churn.value_counts()/df_indefinite.shape[0]

churn
1    0.916396
0    0.083604
Name: count, dtype: float64

Decyzji o pozostaniu mamy niespełna 9%. Można rozważyć takie próbkowanie, aby zwiększyć udział 0. Wtedy jednak nie możemy korzystać z wyników predykcji jak z prawdpodobieństwa. Na tym etapie, spróbujemy stworzyć model, bez zmieniania danych.

In [23]:
df_indefinite.isna().max()

id                             False
is_tv_subscriber               False
is_movie_package_subscriber    False
subscription_age               False
bill_avg                       False
reamining_contract              True
service_failure_count          False
download_avg                   False
upload_avg                     False
download_over_limit            False
churn                          False
is_fixed_term                  False
dtype: bool

In [25]:
# kolumny
df_indefinite.columns

Index(['id', 'is_tv_subscriber', 'is_movie_package_subscriber',
       'subscription_age', 'bill_avg', 'reamining_contract',
       'service_failure_count', 'download_avg', 'upload_avg',
       'download_over_limit', 'churn', 'is_fixed_term'],
      dtype='object')

In [30]:
#wybranie zmiennych do modelu 
x_names = list(df_indefinite.columns[1:-2])

In [32]:
del x_names[x_names.index('reamining_contract')]

In [33]:
# Podział zbioru 

train_x, test_x, train_y, test_y = train_test_split(df_indefinite[x_names],df_indefinite['churn'], test_size=0.3, random_state=123)

In [34]:
# Estymacja modelu
model  = DecisionTreeClassifier().fit(train_x,train_y)

In [35]:
# Ważność zmiennych
model.feature_importances_

array([0.01684852, 0.02915652, 0.28635531, 0.1680284 , 0.02469105,
       0.30725963, 0.15553193, 0.01212864])

In [36]:
# ładniejsza postać
importances = pd.DataFrame(columns=['name','importance'])
importances['name'] = model.feature_names_in_
importances['importance'] = model.feature_importances_
importances.sort_values(by='importance',ascending = False)


,name,importance
5,download_avg,0.307260
2,subscription_age,0.286355
3,bill_avg,0.168028
6,upload_avg,0.155532
1,is_movie_package_subscriber,0.029157
4,service_failure_count,0.024691
0,is_tv_subscriber,0.016849
7,download_over_limit,0.012129


In [44]:
# usuńmy nieistotne zmienne
x_names_2  =importances.loc[importances['importance']>0.05,'name']
x_names_2

2    subscription_age
3            bill_avg
5        download_avg
6          upload_avg
Name: name, dtype: object

In [45]:
train_x2 = train_x[x_names_2]
test_x2 = test_x[x_names_2]

In [46]:
# Estymacja modelu
model_2 = DecisionTreeClassifier().fit(train_x2,train_y)

In [47]:
# Ważność zmiennych
model_2.feature_importances_

array([0.30400808, 0.1858243 , 0.33520629, 0.17496134])

In [49]:
model_2.feature_names_in_

array(['subscription_age', 'bill_avg', 'download_avg', 'upload_avg'],
      dtype=object)

In [50]:
# predykcje 
train_pred = model_2.predict(train_x2)
test_pred = model_2.predict(test_x2)

In [51]:
from sklearn.metrics import classification_report

In [52]:
# classification report train
print(classification_report(train_y,train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1231
           1       1.00      1.00      1.00     13831

    accuracy                           1.00     15062
   macro avg       1.00      1.00      1.00     15062
weighted avg       1.00      1.00      1.00     15062



In [53]:
# classification report test
print(classification_report(test_y,test_pred))

              precision    recall  f1-score   support

           0       0.31      0.32      0.32       568
           1       0.93      0.93      0.93      5888

    accuracy                           0.88      6456
   macro avg       0.62      0.63      0.62      6456
weighted avg       0.88      0.88      0.88      6456



Model jest przeuczony, postaramy się to zmienić zagłębiając się w hiperparametry modelu

# Hiperparametry modelu

Dokumentacja: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

Hiperparametry modelu pomagają zwiększyć jakość modelu oraz zmniejszyć przeuczenie.

Parametry do optymalizacji:
- criterion{“gini”, “entropy”, “log_loss”}, default=”gini” - kryterium optymalizacji drzewa
- splitter{“best”, “random”}, default=”best” - Sposób wyboru najlepszego podziału
- max_depth int, default=None - maksymalna głębokość drzewa
- min_samples_split int or float, default=2 - minimalna liczebność próby, aby dokonać kolejnego podziału
- min_samples_leaf int or float, default=1 - minimalna liczebność próby w liściu.
- min_weight_fraction_leaf float, default=0.0 - minimalny ważony udział z całości zbioru. Jeżeli nie ma wagi, to każdy rekord jest traktowany tak samo.
- max_features int, float or {“sqrt”, “log2”}, default=None - liczba zmiennych rozważana przy każdym podziale.
- random_state - ziarno losowania.
- max_leaf_nodes int, default=None - maksymalna liczba liści
- min_impurity_decrease float, default=0.0 - podział zostanie stworzony, jeżeli wartość criterion zmniejszy się o co najmniej podaną wartość.
- class_weight dict, list of dict or “balanced”, default=None - wagi klas.
- ccp_alpha non-negative float, default=0.0 - parametr złożoności drzewa. dla 0 nie ma przycięć. Im wyższa wartość tym mniej złożone drzewo.
- monotonic_cstarray-like of int of shape (n_features), default=None - ustalenie zależności monotonicznej zmiennych.

In [54]:
# Dodajmy parametry zmniejszające złożoność drzewa
model_2 = DecisionTreeClassifier(max_depth=15, min_samples_leaf=20).fit(train_x2,train_y)

In [55]:
# predykcje 
train_pred = model_2.predict(train_x2)
test_pred = model_2.predict(test_x2)


In [56]:
# classification report train
print(classification_report(train_y,train_pred))

              precision    recall  f1-score   support

           0       0.67      0.26      0.38      1231
           1       0.94      0.99      0.96     13831

    accuracy                           0.93     15062
   macro avg       0.80      0.63      0.67     15062
weighted avg       0.92      0.93      0.91     15062



In [57]:
# classification report test
print(classification_report(test_y,test_pred))

              precision    recall  f1-score   support

           0       0.54      0.20      0.29       568
           1       0.93      0.98      0.95      5888

    accuracy                           0.91      6456
   macro avg       0.73      0.59      0.62      6456
weighted avg       0.89      0.91      0.90      6456



In [58]:
# Dodanie wagi do modelu
model_3 = DecisionTreeClassifier(max_depth=20, min_samples_leaf=25, min_impurity_decrease=0.03, class_weight={1: 1, 0:10}).fit(train_x2,train_y)

In [59]:
# predykcje 
train_pred = model_3.predict(train_x2)
test_pred = model_3.predict(test_x2)

In [60]:
# classification report train
print(classification_report(train_y,train_pred))

              precision    recall  f1-score   support

           0       0.16      0.94      0.27      1231
           1       0.99      0.55      0.70     13831

    accuracy                           0.58     15062
   macro avg       0.57      0.74      0.48     15062
weighted avg       0.92      0.58      0.67     15062



In [61]:
# classification report test
print(classification_report(test_y,test_pred))

              precision    recall  f1-score   support

           0       0.16      0.93      0.28       568
           1       0.99      0.54      0.70      5888

    accuracy                           0.57      6456
   macro avg       0.57      0.73      0.49      6456
weighted avg       0.91      0.57      0.66      6456



Stworzenie modelu z ograniczeniami spowodowało zmniejszenie przeuczenia, jednak jakość predykcji dla 0 jest znacząco niższa. Stąd dodanie wag dla klas, które poprawiło jakość. 